# Review Ground Truth 
This notebook allows fore interactively reviewing data and ground truth labels used for the publication presenting an algorithm to derive respiratory phases in intra-arrest ventilation. 
The data are obtained form an swine experiment of CPR. 

Execute the following cells, than cose from the dropdown menu which case to review. 

In [ ]:
from vitabel import Vitals
from pathlib import Path
import pandas as pd

import ipywidgets as widgets
from IPython.display import display

In [ ]:
def plot_curves(case: Vitals):
    t = case.get_label("cpr").first_entry

    plot = case.plot_interactive(
        channels = [["Flow Interpolated"],
                    ["Pressure Interpolated"],
                    ["Product Flow Pressure"], 
                    ["Product negative Flow Pressures Slope"]],
        labels=[["Inspiration Begin gt", "Inspiration Begin", "Expiration Begin gt", "Expiration Begin", "Inspiration gt", "Expiration gt"] for i in range(0,4)],
        channel_overviews = [["Flow Interpolated"]],
        start = t - pd.Timedelta(0.5,"s"),
        stop = t + pd.Timedelta(7.75,"s"),
        #subplots_kwargs={"figsize":(49,18)},
        subplots_kwargs={
        "figsize": (18, 10), #7.48,
        "dpi" : 100,
        "gridspec_kw": {"height_ratios": [1,1,1,1, 0.5]}},
        time_unit="s")
    return plot

In [ ]:
# Load Data and Generate Plots
cases = {}
widgets_map = {}
for case_path in Path("./data/").glob("*.json"):
    case_id = case_path.stem
    case = Vitals()
    case.load_data(case_path) 
    cases[case_id]=case
    widgets_map[case_id]=plot_curves(case)

In [ ]:
# Generated a Dropdown Menus to swtich betweeen cases
# 1) Dropdown with keys
dd = widgets.Dropdown(
    options=list(widgets_map.keys()),
    value=next(iter(widgets_map)),  # first key as default
    description="Select:"
)
# 2) A container that will hold exactly one widget (the chosen AppLayout)
holder = widgets.Box()
# 3) Callback: swap the child on selection
def on_select(change):
    if change["name"] == "value":
        selected = change["new"]
        holder.children = (widgets_map[selected],)
dd.observe(on_select, names="value")
# 4) Initialize view and display
holder.children = (widgets_map[dd.value],)
display(dd, holder)